In [2]:
from pykdtree.kdtree import KDTree
import time
import trimesh
import numpy as np



In [3]:
from os import sys
sys.path.append("/notebooks/meshAfterParty/")
import skeleton_utils as sk

In [4]:
# this is the proximity limit used in trimesh
import trimesh.proximity as pr
pr.tol.merge

1e-08

In [ ]:
#trimesh example: https://github.com/mikedh/trimesh/issues/153
points = mesh.bounding_box.sample_grid(step=2.0)
results = mesh.ray.contains_points(points)

# test the closest points of trimesh vs. the KDTree

In [5]:
import trimesh
multi_soma_mesh = trimesh.load_mesh("../Platinum_Compartment/multi_soma_example.off")

In [6]:
sig_pieces = sk.split_significant_pieces(multi_soma_mesh,significance_threshold=100)
sig_pieces

[<trimesh.Trimesh(vertices.shape=(571170, 3), faces.shape=(1147084, 3))>,
 <trimesh.Trimesh(vertices.shape=(890, 3), faces.shape=(1776, 3))>,
 <trimesh.Trimesh(vertices.shape=(155, 3), faces.shape=(346, 3))>,
 <trimesh.Trimesh(vertices.shape=(126, 3), faces.shape=(280, 3))>,
 <trimesh.Trimesh(vertices.shape=(53, 3), faces.shape=(118, 3))>]

In [18]:
test_mesh = sig_pieces[1]
main_mesh = sig_pieces[0]
import time

n_vertices = 100

start_time = time.time()
results = pr.closest_point(main_mesh,test_mesh.vertices[:n_vertices])
print(f"Total time for querying with closest_point for {n_vertices} points =  {time.time() - start_time}")

start_time = time.time()
mesh_kd = KDTree(main_mesh.vertices)
kd_results = mesh_kd.query(test_mesh.vertices[:n_vertices])

print(f"Total time for querying with KDtree for {n_vertices} points = {time.time() - start_time}")

Total time for querying with closest_point for 100 points =  0.5038728713989258
Total time for querying with KDtree for 100 points = 0.07745480537414551


In [19]:
start_time = time.time()
main_mesh.ray.contains_points(test_mesh.vertices[:n_vertices])
print(f"time for ray contains for {n_vertices} points = {time.time() - start_time}")

time for ray contains for 100 points = 106.66378283500671


# The signed distance function

In [ ]:
def signed_distance(mesh, points):
    """
    Find the signed distance from a mesh to a list of points.
    * Points OUTSIDE the mesh will have NEGATIVE distance
    * Points within tol.merge of the surface will have POSITIVE distance
    * Points INSIDE the mesh will have POSITIVE distance
    Parameters
    -----------
    mesh   : Trimesh object
    points : (n,3) float, list of points in space
    Returns
    ----------
    signed_distance : (n,3) float, signed distance from point to mesh
    """
    # make sure we have a numpy array
    points = np.asanyarray(points, dtype=np.float64)

    # find the closest point on the mesh to the queried points
    closest, distance, triangle_id = closest_point(mesh, points)

    # we only care about nonzero distances
    nonzero = distance > tol.merge

    if not nonzero.any():
        return distance

    inside = mesh.ray.contains_points(points[nonzero])
    sign = (inside.astype(int) * 2) - 1

    # apply sign to previously computed distance
    distance[nonzero] *= sign

    return distance

# The new filter function

In [20]:
import skeleton_utils as sk

In [ ]:
def filter_away_inside_pieces(
                            main_mesh,
                            pieces_to_test,
                            significance_threshold=2000,
                            n_sample_points=3
                            ):
    significant_pieces = [m for m in mesh_pieces if len(m.faces) > significance_threshold]

    print(f"There were {len(significant_pieces)} pieces found after size threshold")
    if len(significant_pieces) <=0:
        print("THERE WERE NO MESH PIECES GREATER THAN THE significance_threshold")
        return []
    
    final_mesh_pieces = []
    
    for i,mesh in enumerate(significant_pieces):
        #gets the number of samples on the mesh to test (only the indexes)
        idx = np.random.choice(len(mesh.vertices),n_sample_points , replace=False)
        #gets the sample's vertices
        points = mesh.vertices[idx,:]

        start_time = time.time()
        
        #find the signed distance from the sampled vertices to the main mesh
        # Points outside the mesh will be negative
        # Points inside the mesh will be positive
        signed_distance = trimesh.proximity.signed_distance(main_mesh,points)
        
        #gets the 
        outside_percentage = sum(signed_distance <= 0)/n_sample_points
        if outside_percentage > 0.9:
            final_mesh_pieces.append(mesh)
            #print(f"Mesh piece {i} OUTSIDE mesh")
        else:
            #print(f"Mesh piece {i} inside mesh :( ")
            pass
                
    return final_mesh_pieces

In [21]:
list(4)

TypeError: 'int' object is not iterable

In [ ]:
def filter_mesh_significant_outside_pieces(unfiltered_mesh,main_mesh,significance_threshold=2000,n_sample_points=3):
    